In [1]:
from functools import partial
import itertools

In [2]:
class Meta1(type):
    def __new__(mcs, *args, possibles1, **kwargs):
        print("met1",mcs, args, kwargs)
        
        # ----------------------- func
        # > Create a default
        old_f = args[2].get('func', lambda *s, **k: None)
        
        # > Create new
        def func(self, t):
            if t in possibles1:
                return list(map(lambda x: t*x, getattr(self,"aps")))
            else:
                return old_f(self, t)
        
        # > Assign
        args[2]['func'] = func
        
        # -----------------------
        return super().__new__(mcs, *args, **kwargs)

class Meta2(type):
    def __new__(mcs, *args, possibles2, **kwargs):
        print("met2",mcs, args, kwargs)
        
        # ----------------------- func
        # > Create a default
        old_f = args[2].get('func', lambda *s, **k: None)
        
        # > Create new
        def func(self, t):
            if t in possibles2:
                return list(map(lambda x: t+x, getattr(self,"aps")))
            else:
                return old_f(self, t)
        
        # > Assign
        args[2]['func'] = func
        
        # ----------------------- 
        return super().__new__(mcs, *args, **kwargs)

In [3]:
mMeta = type("comb", (Meta1,Meta2),{})
class mClass(metaclass = mMeta, possibles1=[1,2,3],possibles2=[4,5,6]):
    def __init__(self, aps):
        self.aps = aps

met1 <class '__main__.comb'> ('mClass', (), {'__module__': '__main__', '__qualname__': 'mClass', '__init__': <function mClass.__init__ at 0x7f5a90c991f0>}) {'possibles2': [4, 5, 6]}
met2 <class '__main__.comb'> ('mClass', (), {'__module__': '__main__', '__qualname__': 'mClass', '__init__': <function mClass.__init__ at 0x7f5a90c991f0>, 'func': <function Meta1.__new__.<locals>.func at 0x7f5a90c99820>}) {}


In [28]:
a = type(mClass)
a.__mro__

(__main__.comb, __main__.Meta1, __main__.Meta2, type, object)

In [4]:
class FuncList:
    """
    Starts with a list of functions, on call it will apply the argument to the first function,
    then the result to the second and so on
    """

    def __init__(self, funcs):
        self.functions = funcs

    def __call__(self, val):
        r = val
        for i in self.functions:
            r = i(r)
        return r


def gen_func(path): # var1.var2..
    lis = []
    for i,n in enumerate(path.split('.')):
        lis.append(partial(lambda ns,ls: list(itertools.chain(*map(lambda t: getattr(t,ns,None),ls))),n))
    return FuncList(lis)

In [5]:
class retrieve(type):
    def __new__(mcs, *args, direction, **kwargs):
        # ----------------------- get_sub
        # > Create a default
        old_f = args[2].get('get_sub', lambda *s, **k: [])
        
        # > Setup
        
        diz_subs = dict()
        for ln, path in direction.items():
            diz_subs[ln] = gen_func(path)
        
        # > Create new
        def func(self, t):
            if t in diz_subs.keys():
                return diz_subs[t]([self])
            else:
                return old_f(self, t)
        
        # > Assign
        args[2]['get_sub'] = func
        
        # -------------------------
        return super().__new__(mcs, *args, **kwargs)


In [6]:
class C1(metaclass=retrieve, direction={'subs':"subs.store.store", 'down':"down.store"}):
    def __init__(self, subs, down):
        self.subs = subs
        self.down = down

class C3(metaclass=retrieve, direction={'sotto':"sotto.store"}):
    def __init__(self, subs):
        self.sotto = subs
    
class C2:
    def __init__(self, a):
        self.store = a

In [7]:
c1 = C1([C2([C2(["ciao","cia"])]),C2([C2(["Mondo", "World"])])],[C2(["ciao","cia"]),C2(["Mondo", "World"])])
c3 = C3([C2(["Halo"]),C2(["Hola"])])

In [8]:
c1.get_sub('down')

['ciao', 'cia', 'Mondo', 'World']

In [9]:
c3.get_sub('sotto')

['Halo', 'Hola']

In [15]:
issubclass(C1, retrieve)

False

In [22]:
retrieve.__mro__

(__main__.retrieve, type, object)